# Projeto Filtros AdFG

Nesse projeto, visamos comparar diferentes tipos de filtros com relação a diferentes ruídos em uma mesma imagem.

Os filtros propostos são o filtro do kernel da equação do calor e também o de regularização à Tikhonov, este sendo um caso particular de um filtro $ARMA(p,q)$.

Eles são definidos por:

$$\text{heat} (\lambda) = e^{- \tau \lambda}$$

onde $\tau$ é um parâmetro real a ser definido, e

$$\text{Tikhonov} (\lambda) = \frac{1}{1 + \gamma \lambda}$$

onde $\gamma$ é um parâmetro real a ser escolhido. 

Ambos os filtros tem implementação na biblioteca `PyGSP`, com a qual faremos a comparação de desempenho no final do trabalho.

## Imports

In [5]:
pip install opencv-python


   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.6 MB 5.7 MB/s eta 0:00:07
    --------------------------------------- 0.6/38.6 MB 8.1 MB/s eta 0:00:05
   - -------------------------------------- 1.0/38.6 MB 7.2 MB/s eta 0:00:06
   - -------------------------------------- 1.7/38.6 MB 8.4 MB/s eta 0:00:05
   -- ------------------------------------- 2.5/38.6 MB 9.2 MB/s eta 0:00:04
   ---- ----------------------------------- 3.9/38.6 MB 10.0 MB/s eta 0:00:04
   ---- ----------------------------------- 4.6/38.6 MB 10.2 MB/s eta 0:00:04
   ----- ---------------------------------- 5.7/38.6 MB 10.4 MB/s eta 0:00:04
   ------ --------------------------------- 6.1/38.6 MB 10.2 MB/s eta 0:00:04
   ------- -------------------------------- 7.1/38.6 MB 10.8 MB/s eta 0:00:03
   ------- -------------------------------- 7.7/38.6 MB 10.7 MB/s eta 0:00:03
   --------- ------------------------------ 8.7/38.6 MB 10.9 MB/s eta 0:00:03

In [1]:
# bibliotecas 
import numpy as np 
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting 
from PIL import Image, ImageFilter
import scipy.sparse as sp
import cv2 # para mudar o tamanho das fotos

In [2]:
# abrindo imagens
img_original = cv2.imread('cameraman_original.jpg')
img_ruido1 = cv2.imread('cameraman_ruido.jpg')
img_ruido2 = cv2.imread('cameraman_ruido2.jpg')

print("Tamanho original: ", img_original.shape[:2])
print("Tamanho ruido 1: ", img_ruido1.shape[:2])
print("Tamanho ruido 2: ", img_ruido2.shape[:2])

Tamanho original:  (256, 256)
Tamanho ruido 1:  (256, 256)
Tamanho ruido 2:  (256, 256)


### Reduzindo as imagens

In [3]:
img_original = cv2.resize(img_original, (120,120))
img_ruido1 = cv2.resize(img_ruido1, (120,120))
img_ruido2 = cv2.resize(img_ruido2, (120,120))

print("Tamanho original: ", img_original.shape[:2])
print("Tamanho ruido 1: ", img_ruido1.shape[:2])
print("Tamanho ruido 2: ", img_ruido2.shape[:2])

Tamanho original:  (120, 120)
Tamanho ruido 1:  (120, 120)
Tamanho ruido 2:  (120, 120)


In [4]:
# transformando em escala de cinza
img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
img_ruido1 = cv2.cvtColor(img_ruido1, cv2.COLOR_BGR2GRAY)
img_ruido2 =  cv2.cvtColor(img_ruido2, cv2.COLOR_BGR2GRAY)

### Transformando em sinal

In [5]:
sinal_original = np.array(img_original).reshape(120**2)
sinal_ruido1 = np.array(img_ruido1).reshape(120**2)
sinal_ruido2 = np.array(img_ruido2).reshape(120**2)

## Implementação dos filtros e das métricas 

Abaixo, estão definidas as funções utilizadas para calcular os filtros Heat e Tikhonov, bem como as métricas de Erro Quadrático Médio (MSE) e Peak Signal-to-Noise Ratio (PSNR).

Na ordem:

* Heat
* Tikhonov
* MSE
* PSNR

In [6]:
def heat(lam,tau=1):
    '''Calcula a função do filtro a partir de um vetor de autovalores.
    
    Parâmetros:
    - lambda (array) - vetor de autovalores
    
    Return:
    - H (array) - matriz diagonal cujos calores da diagonal são os h(lambda_i) e
    h() é a função que define o filtro
    '''
    # primeiro definir a função 
    def h(x,tau = tau):
        return np.exp(- tau * x)
    
    h_lambda = h(lam,tau)
    H = np.diag(h_lambda)

    return H


In [7]:
def tikhonov(lam,g=1):
    '''Calcula a matriz H da regularização à tikhonov a partir de
    um vetor de autovalores.
    
    Parâmetros:
    - lam (array): vetor de autovalores
    - g (float): default = 1, valor do parâmetro lambda 
    
    Returns:
    - H (array): matriz diagonal cujas entradas são a função h() 
    calculada sobre os autovalores correspondentes.'''
    
    def h(x,g=g):
        return 1/(1+ g*x)
    
    h_lambda = h(lam)
    H = np.diag(h_lambda)

    return H 

In [8]:
def MSE(original,filtrado):
    '''Calcula o erro quadrático médio da filtração escolhida.
    
    Parâmetros:
    - original (array): sinal original, sem ruídos, para usar como
    referência no MSE
    - filtrado (array): sinal obtido com a filtração
    
    Returns:
    - mse (float): erro quadrático médio'''

    soma = sum((original - filtrado)**2) # soma dos quadrados 
    n = len(filtrado)
    mse = soma/n
    return mse

In [9]:
def PSNR(original,filtrado,img=True):
    '''Calcula o Peak signal-to-noise ratio com relação a alguma fltração
    escolhida. O PSNR é a variação relativa do máximo valor possível
    do sinal em relação ao erro quadrático médio (norma L2 do erro),
    em escala logarítmica.
    
    Parâmetros:
    - original (array): sinal original
    - filtrado (array): sinal filtrado de acordo com a filtração de 
    escolha
    
    Returns: 
    - res (float): medida do PSNR'''
    mse = MSE(original,filtrado) # precisa dessa métrica
    if img:
        max_f = 255
    else: 
        # definindo uma métrica alternativa para meu toy problem
        max_f = max(filtrado)
    res = 20*np.log10(max_f) - 10*np.log10(mse)
    return res

## Grafo Clássico

Nessa seção, vamos fazer todas as contas para os filtros implementados aqui com o grafo construído de maneira clássica, isto é, conectando os pixels adjascentes.

Além disso, vamos dividir os resultados entre os filtros Heat e Tikhonov. 

In [10]:
# primeiro a identificação das bordas
def bordas(img):
    '''Função que recupera os índices de cada uma das bordas de um array (ou imagem).
    
    Parâmetros:
    - img (array): matriz mxn que representa a imagem (ou qualquer outra coisa)
    
    Return:
    - left (list): índices da borda esquerda 
    - right (list): índices da borda direita
    - up (list): índices da borda superior
    - down (list): índices da borda inferior'''
    # matriz m por n 
    m, n = img.shape 

    left = [i*m for i in range(n)]
    right = [i*m - 1 for i in range(1,n+1)]
    up = [i for i in range(m)]
    down = [n*(m-1) + i for i in range(m)]

    return left, right, up, down

In [11]:
def grafo_img_simples(img, adj = False, plot=True):
    '''Função que retorna uma instância do objeto graph do pygsp que
    representa uma imagem. Aqui o grafo é o mais simples possível, considerando
    adjascentes os pixels vizinhos.
       
    Parâmetros:
     - img (array): imagem a partir da qual construiremos o grafo 
     - adj (Bool): default = False, indica se retornamos ou não a matriz de adjascência
      do grafo criado 
     - plot (Bool): default = True, faz o desenho do grafo
    '''
    # primeira coisa é pegar as bordas 
    left, right, up, down = bordas(img)
    m,n = img.shape
    #W = np.zeros((m*n,m*n))
    W = []

    for i in range(m*n):
        linha = np.zeros(m*n,dtype=np.int16)

        if i == 0: # canto superior esquerdo 
            linha[1] = 1 # a direita
            linha[m] = 1 # abaixo
            linha[m+1] = 1 # diagonal p baixo

        elif i == m-1: # canto superior direito
            linha[i-1] = 1 # a esquerda 
            linha[i+m] = 1 # abaixo 
            linha[i+m-1] = 1 # diagonal p baixo

        elif i == n*(m-1): # canto inferior esquerdo 
            linha[i+1] = 1 # a direita
            linha[i-m] = 1 # acima
            linha[i-m+1] = 1

        elif i == n*m -1: # canto inferior direito
            linha[i-1] = 1 # a esquerda
            linha[i-m] = 1 # acima
            linha[i-m-1] = 1

        elif i in up:
           linha[i-1] = 1
           linha[i+1] = 1
           linha[i+m] = 1
           linha[i+m-1] = 1
           linha[i+m+1] = 1
            
        elif i in left:
            linha[i+1] = 1
            linha[i-m] = 1
            linha[i+m] = 1
            linha[i-m+1] = 1
            linha[i+m+1] = 1
        
        elif i in right:
            linha[i-m] = 1
            linha[i+m] = 1
            linha[i-1] = 1
            linha[i-m-1] =1
            linha[i+m-1] = 1

        elif i in down:
            linha[i-1] - 1
            linha[i+1] = 1
            linha[i-m] = 1
            linha[i-m+1] = 1
            linha[i-m-1] = 1

        else:
            linha[i-m-1] = 1
            linha[i-m] = 1
            linha[i-m+1] = 1
            linha[i-1] = 1
            linha[i+1] = 1
            linha[i+m-1] = 1
            linha[i+m] = 1
            linha[i+m+1] = 1
        
        W.append(linha)
    W = sp.csc_matrix(np.array(W, dtype=np.int16))
    #assert W.shape == (m*n,m*n)
    
    # criando o grafo 
    G = graphs.Graph(W)
    
    if adj:
        return G, W
    elif plot:
        G.set_coordinates()
        G.plot(title=f"Grafo trivial para imagem {m}x{n}")
        return G
    else:
        return G

Note que essa construção independe do sinal, então vamos fazê-la a partir da imagem original e computar somente os filtros a partir das imagens com ruido.

In [12]:
G_classico = grafo_img_simples(img_original, False, False)

### Heat

In [13]:
# calculando autofunções
G_classico.compute_fourier_basis()

2024-05-15 20:38:51,260:[WARNING](pygsp.graphs.graph.compute_fourier_basis): Computing the full eigendecomposition of a large matrix (14400 x 14400) may take some time.


In [14]:
U = G_classico.U # autofunções
lam = G_classico.e # autovalores

In [15]:
# calculando a filtração 
tau = [1/4, 1/2, 1, 2, 4]

heat_ruido1 = []
heat_ruido2 = []
for t in tau:
    r1 = U @ heat(lam, tau = t) @ U.T @ sinal_ruido1
    r2 = U @ heat(lam, tau = t) @ U.T @ sinal_ruido2
    heat_ruido1.append(r1)
    heat_ruido2.append(r2)

### Tikhonov

## Grafo com Pesos

Nessa seção, introduzimos um novo método para construção dos grafos, em que é aplicado um peso Gaussiano quando dois vértices são $\kappa$-próximos, ou seja, 

$$a_{ij} = e^{- d^2 / 2 \sigma^2}, \text{se} |d(i,j)| < \kappa$$

A distância aqui utilizada é a diferença entre as cores de cada pixel em grayscale.

In [35]:
def grafo_peso_grayscale(img,th,kap,adj=False,plot=True):
    '''Função que constrói o grafo para uma imagem em escala de cinzas
    a partir do gradiente de cor. Os pesos atribuidos a cada vértice serão
    uma gaussiana de variância theta^2 avaliada em em x = gray(i) - gray(j), 
     onde gray() é o valor na escala de cinzas de cada pixel.
      
    Parâmetros:
     - img (array): matrix representando a imagem, suas entradas correspondem ao valor
     de cada vértice na escala de cinza
     - th (float): desvio padrão da Gaussiana
     - kap (float): threshold de conexão dos vértices'''
    
    m,n = img.shape
    N = m*n
    W = np.zeros((N,N), dtype=np.int8)
    #W = sp.coo_matrix(shape=(N,N),dtype=np.float16)
    for i in range(N):
        for j in range(N):
            # já que estou iterando num vetor dos vértices, preciso recuperar o índice
            ii = i % n 
            ij = i % m 

            ji = j % n 
            jj = j % m
            # diferença de grayscale
            dist = img[ij,ii] - img[jj,ji]
            if abs(dist) <= kap:
                W[i,j] = np.exp(- dist*dist / (2*th*th), dtype=np.float32)
    
    G = graphs.Graph(W)
    if adj:
        return G, W
    elif plot:
        G.set_coordinates()
        G.plot(title=f"Grafo com pesos $\kappa =$ {kap}")
        return G
    else:
        return G
    

Diferentemente do primeiro grafo, essa construção depende do valor do sinal para calcular as distâncias entre os vértices, então temos um grafo para cada ruído diferente.

In [36]:
G_ruido1_peso = grafo_peso_grayscale(img_ruido1, 1, 5)
G_ruido2_peso = grafo_peso_grayscale(img_ruido2, 1, 5)

C:\Users\mevel\AppData\Local\Temp\ipykernel_11532\2024869093.py:28: RuntimeWarning: overflow encountered in scalar negative
  W[i,j] = np.exp(- dist*dist / (2*th*th))
C:\Users\mevel\AppData\Local\Temp\ipykernel_11532\2024869093.py:28: RuntimeWarning: overflow encountered in scalar multiply
  W[i,j] = np.exp(- dist*dist / (2*th*th))
C:\Users\mevel\AppData\Local\Temp\ipykernel_11532\2024869093.py:26: RuntimeWarning: overflow encountered in scalar subtract
  dist = img[ij,ii] - img[jj,ji]


KeyboardInterrupt: 

### Heat

### Tikhonov

## Comparação com o PyGSP